### League of Legends: Spring 2020 Dataset

This datasets contains all games across differnt professional League of Legends 

In [1]:
#import packages
import pandas as pd
import requests
import json
import pickle
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import numpy as np
import pdb

In [2]:
#----Read competitve history data set for list of patches ----#
patches = pd.read_csv('../data/2020 spring match data OraclesElixir 2020-05-15.csv')
patches = list(set(patches['patch']))
patches = [str(x) for x in patches]

In [3]:
def reformat_patch_number(patch):
    patch_split = patch.split('.')
    patch_split = [str(int(x)) for x in patch_split]
    patch_output = patch_split[0] + '.' + patch_split[1] + '.1'
    return(patch_output)
    

In [4]:
patches = [reformat_patch_number(x) for x in patches]

In [5]:
#----- In case we need to use API ------#

#One time api key creation
file_name = '/Users/horacefung/Documents/keys/lol_api_key'
#key = {'key':''} #don't show key
#output = open(file_name,'wb')
#pickle.dump(key, output)
#output.close()

input_file = open(file_name,'rb')
api_key = pickle.load(input_file)['key']
input_file.close()

#api_root_url = "https://www.googleapis.com/youtube/v3/videos?"
#url_params = "part=snippet&type=video&chart=mostPopular"

#url = api_root_url + url_params + api_key
#data = requests.get(url).json()

In [6]:
#----- Pull champion list -----#
url = 'http://ddragon.leagueoflegends.com/cdn/' + patches[-1] + '/data/en_US/champion.json'
data = requests.get(url).json()
champion_list = list(data['data'].keys())

In [25]:
def extract_data(champion, champion_name, patch):
    
    #Initialize porter object
    porter = PorterStemmer()
    
    #Split into base stats & spells
    stats = champion['stats']
    passive = champion['passive']
    spells = champion['spells']
    
    #Keywords
    hard_cc_list = ['airborne', 'charm', 'flee', 'taunt', 'sleep', 'stun', 'supression', 'suspension', 
                    'stasis', 'pull', 'knock']
    soft_cc_list = ['blind', 'cripple', 'disarm', 'ground', 'knockdown', 'nearsight', 'root', 'silence', 'slow']
    gap_closer_list = ['dash', 'blink', 'leap', 'launch', 'movementspeed', 'teleport']
    vision_list = ['vision', 'sight']
    protection_list = ['shield', 'heal']
    
    #initialize variables
    hard_cc_value = 0
    soft_cc_value = 0
    spells_average_range_value = 0
    gap_closer_value = 0
    protection_value = 0
    
    #Passive
    passive = passive['description']
    passive = passive.lower()
    passive = passive.replace('movement speed', 'movementspeed') #specific logic for move speed
    passive = [porter.stem(x) for x in word_tokenize(passive)]
    hard_cc = len(list(set(passive) & set(hard_cc_list))) #unique key words
    soft_cc = len(list(set(passive) & set(soft_cc_list))) #unique key words
    protection = len(list(set(passive) & set(protection_list)))
    gap_closer = len(list(set(passive) & set(gap_closer_list)))
    
    #Update talley with passive
    hard_cc_value = hard_cc_value + hard_cc
    soft_cc_value = soft_cc_value + soft_cc
    protection_value = protection_value + protection
    gap_closer_value = gap_closer_value + gap_closer
    
    
    #Four spells
    for i in range(4):
        
        #----Extract text information from tooltip
        tooltip = spells[i]['tooltip'].lower()
        tooltip = tooltip.replace('movement speed', 'movementspeed') #specific logic for move speed
        tooltip = [porter.stem(x) for x in word_tokenize(tooltip)]
        hard_cc = len(list(set(tooltip) & set(hard_cc_list))) #unique key words
        soft_cc = len(list(set(tooltip) & set(soft_cc_list))) #unique key words
        protection = len(list(set(tooltip) & set(protection_list)))
        gap_closer = len(list(set(tooltip) & set(gap_closer_list)))
        spells_range = np.mean(spells[i]['range'])
        
        #Update talley
        hard_cc_value = hard_cc_value + hard_cc
        soft_cc_value = soft_cc_value + soft_cc
        protection_value = protection_value + protection
        gap_closer_value = gap_closer_value + gap_closer
        spells_average_range_value = np.mean([spells_average_range_value, spells_range])
    
    #-----Setup Dataframe---------#
    dict_temp = {'champion' : champion_name,
                 'patch' : patch,
                 'hard_cc_value' : hard_cc_value, 
                 'soft_cc_value' : soft_cc_value,
                 'spells_average_range_value' : spells_average_range_value,
                 'gap_closer_value' : gap_closer_value,
                 'protection_value' : protection_value}
    
    dict_temp = {**stats, **dict_temp}
    
    output_df = pd.DataFrame(dict_temp, index = [0])
    output_df = output_df.set_index(['champion', 'patch'])
    
    
    return(output_df)

In [30]:
#----- Pull Relevant Champion Patch Data ---------#
def data_pull(patches, champion_list):
    
    champion_output = pd.DataFrame()
    loops = len(patches)
    counter = 0
    
    for patch in patches:
        
        for champion in champion_list:
            url = 'http://ddragon.leagueoflegends.com/cdn/{}/data/en_US/champion/{}.json'.format(patch, champion)
            
            try:
                data = requests.get(url).json()
                data = data['data'][champion]
                champion = extract_data(data, champion, patch)
                champion_output = pd.concat([champion_output, champion])
                print('Added:' + champion)
                print(champion)
            except:
                continue
                #print('Request failed')
                #do nothing if champion not in this patch, e.g. new releases
        
        counter = counter + 1
        print('Completed patch: ' + patch + ' | ' + str(counter) + '/' + str(loops))
    
    return(champion_output)

In [33]:
champion_dataset = data_pull(patches, champion_list)

Completed patch: 10.4.1 | 1/8
Completed patch: 10.6.1 | 2/8
Completed patch: 10.1.1 | 3/8
Completed patch: 10.2.1 | 4/8
Completed patch: 10.3.1 | 5/8
Completed patch: 10.5.1 | 6/8
Completed patch: 10.7.1 | 7/8
Completed patch: 10.8.1 | 8/8


In [34]:
champion_dataset.head()

,,armor,armorperlevel,attackdamage,attackdamageperlevel,attackrange,attackspeed,attackspeedperlevel,crit,critperlevel,gap_closer_value,...,movespeed,mp,mpperlevel,mpregen,mpregenperlevel,protection_value,soft_cc_value,spellblock,spellblockperlevel,spells_average_range_value
champion,patch,,,,,,,,,,,,,,,,,,,,,
Ezreal,10.4.1,22.00,3.50,60.00,2.500,550,0.625,2.50,0,0,1,...,325,375.0,50.0,8.50,0.65,0,0,30.0,0.50,12834.3750
Leona,10.4.1,47.00,3.60,60.04,3.000,125,0.625,2.90,0,0,1,...,335,302.2,40.0,6.00,0.80,0,2,32.1,1.25,881.2500
Mordekaiser,10.4.1,37.00,3.00,61.00,4.000,175,0.625,1.00,0,0,0,...,335,100.0,0.0,0.00,0.00,2,1,32.0,1.25,3667.1875
Ornn,10.4.1,36.00,4.00,69.00,3.500,175,0.625,2.00,0,0,0,...,335,340.6,45.0,8.01,0.60,0,3,32.1,1.25,4537.5000
Veigar,10.4.1,22.55,3.75,50.71,2.625,550,0.625,2.24,0,0,0,...,340,490.0,26.0,8.00,0.80,0,0,30.0,0.50,675.0000


In [35]:
len(champion_dataset)

1184

In [36]:
#---- Pickle Patch Data -----#
file_name = '../data/patch_notes.pickle'
#output = open(file_name,'wb')
#pickle.dump(champion_dataset, output)
#output.close()

input_file = open(file_name,'rb')
patch_data = pickle.load(input_file)
input_file.close()
